In [1]:
# Import libraries
import os
import requests
import time
import pandas as pd
import geopandas as gpd
import ast
from pandas.io.json import json_normalize
import json
from shapely.geometry import Point

# My config.py simply stores my API credentials
# Create one by writing in a single line: key = 'insert_your_google_maps_API_key_here'
import config_googlemapsapi

# This is a file that contains a class and methods to work with the Google Places API (ggplacesapi.py)
import ggplacesapi

In [2]:
# Set parameter values
key = config_googlemapsapi.key

# Initialize GooglePlaces class
api = ggplacesapi.GooglePlaces(key)

# NOTE: I sought to obtain 
# Sampling points in Knox County to bypass API restrictions; systematic sampling in an area (Knox County in this instance)
# In ArcGIS, I created a fishnet of sample points 2500m apart from each other
# There is a python script, 'geospatialgrid.py' that creates your own grid
samples = pd.read_csv("../Data/KnoxSamplePoints.csv", usecols = ['DDLat', 'DDLon'])

In [4]:
# Fields to be retained
fields = ["geometry", "vicinity", "id", "name", "opening_hours", "photos", "place_id", "plus_code", "rating", "reference", "types", "user_ratings_total"]

# Initialize parameters for Google Maps Places API requests
# radius = 1500 # meters; 16093.4 is equivalent to 10 miles; radius in this instance is less than 1 mile.
rankby = 'distance'
keyword = 'store' #'grocery'
type = 'convenience_store'

# Lists that store locations with 60 results; this indicates that we should offset the location and search nearby areas to get a comprehensive set of places
#resample = pd.DataFrame(columns = ["DDLat", "DDLon1", "DDLon2"])
resample_lat = []
resample_lon = []
resample_type = []
counter = 0

# Instantiate a data frame where all POIs will be stored
df = pd.DataFrame(columns = fields)

for x in range(len(samples)):

    # Location to be searched
    lat = samples.loc[x, 'DDLat']
    lon = samples.loc[x, 'DDLon']
    location = str(lat) + ',' + str(lon)


    # Measure of progress
    if x % 100 == 0:
        print("On sample:", x)#, "and type:", poitype)

    # Search and store results in a list
    # Returns all places of a particular type within a specified radius of the location
    places = api.search_places_by_coords(location, keyword, type, rankby)

    # If there are no POIs of a particular type within the specified radius of the location, skip to the next location
    if len(places) == 0:
        next

    else:
        pois = pd.DataFrame(places)

        for name in range(len(fields)):
            if fields[name] not in pois.columns:
                pois.loc[:, fields[name]] = None

        pois = pois[["geometry", "vicinity", "id", "name", "opening_hours", "photos", "place_id", "plus_code", "rating", "reference", "types", "user_ratings_total"]]
        pois = pois.assign(sampleloc = location)
        df = df.append(pois)

        if len(places) >= 60:
            resample_lat.append(lat)
            resample_lon.append(lon)
    
#resample.loc[x, 'DDLon1'] = lon + 0.011228939999988086 # Half of the difference between two locations in degrees
#resample.loc[x, 'DDLon2'] = lon + -0.011228939999988086
resample = pd.DataFrame({'Latitude': resample_lat, 'Longitude': resample_lon})
resample.to_csv("Data/Resample_Locs_20220321.csv", mode = "a", header=False)

# Export to csv
df.to_csv("../Data/Knox_County_Convenience_20220321_Unsimplified.csv", mode='a', header=False)
len(df)

On sample: 0
On sample: 100
On sample: 200
On sample: 300
On sample: 400


25336

In [5]:
# Drop duplicate places
df2 = df.drop_duplicates(subset="place_id", keep="first")

# Reset index after dropping duplicates
df2 = df2.reset_index(drop=True)
#df.to_csv("Data/DataRetrieval/Knox_County_Grocery_20220219.csv", mode='a', header=False)
# Drop 'unnamed' column, if necessary
#pois = pois.drop("Unnamed: 0", axis=1)

# Flatten nested dictionary for geometry
geomPOI = json_normalize(df2["geometry"]) # If dictionary 
#geomPOI = df["geometry"].apply(lambda x: ast.literal_eval(x)) # If string, step 1
#geomPOI = geomPOI.apply(pd.Series) # If string, step 2

# Concatenate latitude and longitude of POIs to data-frame and drop the geometry column
df2 = pd.concat([df2, geomPOI[["location.lat", "location.lng"]].apply(pd.Series)], axis=1).drop("geometry", axis = 1)

# Export to csv
df2.to_csv("../Data/Knox_County_Convenience_20220321.csv", header=True)

C:\Users\jfeng\Anaconda3\envs\GooglePlaces\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


In [15]:
df2 = pd.read_csv("../Data/Knox_County_Grocery_20220222.csv", index_col=0, header=None)
len(df2)

341

In [29]:
# Manual verification/ground truthing of store types
# Find all rows and their indices which are not grocery stores

non_grocery_store_ids = [0, 2, 10, 23, 42, 43, 47, 53, 58, 60, 61, 62, 73, 75, 78, 79, 81, 83, 86, 90, 92, 93, 94, 96, 97, 99, 101, 105, 106, 110,
 111, 113, 114, 116, 117, 118, 120, 121, 122, 123, 124, 128, 130, 131, 132, 133, 135, 137, 140, 141, 144, 145, 147, 149, 151,
 160, 161, 162, 163, 164, 165, 166, 169, 170, 172, 175, 177, 189, 190, 191, 192, 195, 196, 202, 211, 212, 214, 215, 216, 217,
 219, 221, 222, 229, 231, 232, 235, 237, 239, 240, 241, 242, 244, 245, 246, 247, 251, 252, 253, 254, 256, 258, 259, 260, 261, 262, 265, 267, 268, 270, 272,
 275, 276, 282, 288, 289, 290, 292, 293, 294, 296, 298, 301, 302, 303, 304, 305, 306, 307, 309, 313, 314, 315, 316, 317, 326,
 327, 328, 331, 332, 334, 335, 338, 339]

# 0: Gas station
# 2: Ingles gas station
# 10: Shell gas station
# 23: Frozen meats only; not sure if it even exists now. Google Streets View suggests the building it was in is up for sale now.
# 42: Not a grocery store
# 43: BP gas station
# 47: Marathon gas station
# 53: Gas station
# 58: Shell gas station
# 60: Marathon gas station
# 61: ATM
# 62: BP gas station
# 73: Mexican restaurant
# 75: Convenience store
# 78: Convenience store
# 79: Gas station
# 81: Convenience store
# 83: ATM
# 86: Not a grocery store
# 90: Convenience/alcohol store
# 92: Convenience store
# 93: Convenience store
# 94: Convenience store
# 96: No grocery store
# 97: Convenience store/Deli
# 99: Not a grocery store
# 101: Not a grocery store
# 105: Convenience store/Deli
# 106: No grocery store
# 110: Convenience store/Deli
# 111: Convenence store/Deli
# 113: Not a grocery store
# 114: Bread store
# 116: Not a grocery store
# 117: Not a grocdery store
# 118: Convenience store
# 120: Convenience store/Deli
# 121: Convenience store
# 122: ATM
# 123: Gas station store
# 124: Gas station store
# 128: Gas station
# 130: Not a grocery store
# 131: Restaurant
# 132: Not a grocery store
# 133: Not a grocery store
# 135: Convenience store
# 137: Not a grocery store
# 140: Gas station
# 141: Floral store
# 144: Neighborhood
# 145: Bread store
# 147: Not a grocery store
# 149: Convenience store/Deli
# 151: Gas station
# 160: Pharmacy
# 161: Not a grocery store
# 162: Not a grocery store
# 163: Furniture store
# 164: ATM
# 165: Not a grocery store
# 166: Not a grocery store
# 169: Not a grocery store
# 170: Restaurant
# 172: Convenience store
# 175: Deli
# 177: Deli
# 189: Not a grocery store
# 190: Not a grocery store
# 191: Restaurant
# 192: Not a grocery store
# 195: Not a grocery store
# 196: Not a grocery store
# 202: Not a grocery store
# 211: Not a grocery store
# 212: Not a grocery store
# 214: Not a grocery store
# 215: Convenience store
# 216: Not a grocery store
# 217: Not a grocery store
# 219: Not a grocery store
# 221: Convenience store
# 222: ATM
# 229: Convenience store
# 231: ATM
# 232: Not a grocery store
# 235: Bread store
# 237: Convenience store
# 239: Convenience store
# 240: Not a grocery store
# 241: Convenience stoer
# 242: Convenience store
# 244: Not a grocery store
# 245: Convenience store
# 246: Not a grocery store
# 247: convenience store
# 251: Not a grocery store
# 252: Convenience store
# 253: Convenience store
# 254: Convenience store
# 255: Not a grocery store
# 256: Gas station
# 258: Convenience store
# 259: Restaurant
# 260: Restaurant
# 261: Not a grocery store
# 262: Not a grocery store
# 265: Not a grocery store
# 267: Not a grocery store
# 268: Convenience store
# 270: Convenience store
# 272: Convenience store
# 275: Not a grocery store
# 276: Gas station
# 282: Not a grocery store
# 288: Not a grocery store
# 289: Convenience store
# 290: Bread store
# 292: Convenience store
# 293: Not a grocery store
# 294: Bakery
# 296: Not a grocery store
# 298: Gas station
# 301: Not a grocery store
# 302: Convenience store
# 303: Convenience store
# 304: Not a grocery store
# 305: Furniture store
# 306: Not a grocery store
# 307: Convenience store
# 309: Convenience store
# 313: Not a grocery store
# 314: Conveinece store
# 315: Convenience store
# 316: Convenience store
# 317: Not a grocery store
# 326: Convenience store
# 327: Not a grocery store
# 328 Convenience store
# 331: Not a grocery store
# 332: Convenience store
# 334: Convenience store
# 335: Convenience store
# 339: Not a grocery store
# 338: Fireworks store

In [34]:
# Filter rows to only those that are grocery stores

# Filter by index (based on ground truthing by reviewing and verifying each location online)
grocerystores = df2[~df2.index.isin(df2.loc[non_grocery_store_ids].index)]
# Filter by POI type
grocerystores = grocerystores[~pd.DataFrame(grocerystores.types.tolist()).isin(["gas_station", "convenience_store"]).any(1).values]
grocerystores = grocerystores.reset_index(drop=True)
grocerystores.to_csv("../Data/Knox_County_Grocery_20220222_v2_Cleaned.csv", header=True)

In [ ]:
# Export addresses (POI Name + Address) to a text file to be used as search strings in Google Maps
addresses = grocerystores["name"] + ", " + grocerystores["vicinity"]
addresses.to_csv('../Data/Grocery_Addresses_20220222.txt', header=False, index=False, sep='\n', mode='w')
addresses

In [30]:
from ast import literal_eval
convenience = pd.read_csv("../Data/Knox_County_Grocery_20220222.csv", index_col=0, converters={'types': literal_eval})
convenience.head()

,vicinity,id,name,opening_hours,photos,place_id,plus_code,rating,reference,types,user_ratings_total,sampleloc,location.lat,location.lng
0,"3108 Ralph Phelps Rd, Louisville",NaN,Big Al's market,{'open_now': True},"[{'height': 2048, 'html_attributions': ['<a hr...",ChIJMZqfYQooXIgRUz5mOEOspps,"{'compound_code': 'QVQX+Q4 Louisville, Tenness...",4.2,ChIJMZqfYQooXIgRUz5mOEOspps,"[grocery_or_supermarket, food, point_of_intere...",35,"35.80291456,-84.17240950999998",35.789467,-84.102203
1,"1210 E Broadway St, Lenoir City",NaN,United Grocery Outlet,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJAx14s7gqXIgRw93A9No_iuQ,"{'compound_code': 'QPXW+J6 Lenoir City, Tennes...",4.3,ChIJAx14s7gqXIgRw93A9No_iuQ,"[grocery_or_supermarket, supermarket, food, po...",160,"35.80291456,-84.17240950999998",35.799037,-84.254427
2,Farragut,NaN,The I Market,{'open_now': True},NaN,ChIJdwqTGKUuXIgR7urjuq4tsUA,"{'compound_code': 'VRFF+R9 Farragut, Tennessee...",4.6,ChIJdwqTGKUuXIgR7urjuq4tsUA,"[grocery_or_supermarket, food, point_of_intere...",7,"35.80291456,-84.17240950999998",35.874589,-84.176555
3,"11847 Kingston Pike, Knoxville",NaN,Ingles Markets,{'open_now': True},"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJZT52r6UuXIgRDBYyqkCFc2I,"{'compound_code': 'VRFC+WR Knoxville, Tennesse...",4.3,ChIJZT52r6UuXIgRDBYyqkCFc2I,"[grocery_or_supermarket, pharmacy, bakery, sup...",761,"35.80291456,-84.17240950999998",35.874789,-84.177980
4,"404 US-321 #1, Lenoir City",NaN,Ingles Markets,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJTbfmsckqXIgRHuof46JGm-E,"{'compound_code': 'RP2P+RV Lenoir City, Tennes...",4.4,ChIJTbfmsckqXIgRHuof46JGm-E,"[supermarket, pharmacy, bakery, grocery_or_sup...",1054,"35.80291456,-84.17240950999998",35.802097,-84.262802


In [31]:
# Filter rows to only those that are grocery stores

# Filter by index (based on ground truthing by reviewing and verifying each location online)
#convenience = convenience[~convenience.index.isin(convenience.loc[non_grocery_store_ids].index)]
# Filter by POI type
convenience = convenience[~pd.DataFrame(convenience.types.tolist()).isin(["grocery_or_supermarket", "supermarket", "bakery"]).any(1).values]
convenience = convenience.reset_index(drop=True)
convenience.to_csv("../Data/Knox_County_Convenience_20220321_v2_Cleaned.csv", header=True)

In [33]:
convenience

,vicinity,id,name,opening_hours,photos,place_id,plus_code,rating,reference,types,user_ratings_total,sampleloc,location.lat,location.lng
0,"310 13th St, Knoxville",NaN,Go Vols 13th St. Grocery & Deli,{'open_now': True},"[{'height': 1366, 'html_attributions': ['<a hr...",ChIJTR7H6t4XXIgRq0GY97_m58w,"{'compound_code': 'X36C+MP Knoxville, Tennesse...",5.0,ChIJTR7H6t4XXIgRq0GY97_m58w,"[convenience_store, food, point_of_interest, s...",2,"35.83933442,-83.90291492",35.961717,-83.928165
1,"2539 Cecil Ave, Knoxville",NaN,Doan's Market & Deli,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJi6a8nEEWXIgRZ-CDG9LFfrM,"{'compound_code': 'X3WV+WJ Knoxville, Tennesse...",4.0,ChIJi6a8nEEWXIgRZ-CDG9LFfrM,"[convenience_store, food, point_of_interest, s...",48,"35.83933442,-83.90291492",35.997371,-83.905895
2,"612 Tipton Station Rd, Knoxville",NaN,King's Market & Deli,{'open_now': True},NaN,ChIJrU_7jysaXIgReMU7LUehuVc,"{'compound_code': 'V5R4+MW Knoxville, Tennesse...",5.0,ChIJrU_7jysaXIgReMU7LUehuVc,"[food, point_of_interest, store, establishment]",1,"35.83933442,-83.85799916",35.891702,-83.842638
3,"8119 Chapman Hwy, Knoxville",NaN,Pioneer Market,{'open_now': True},NaN,ChIJ2ev5-EYaXIgRKGTHQwaIzPY,"{'compound_code': 'W52G+7R Knoxville, Tennesse...",1.5,ChIJ2ev5-EYaXIgRKGTHQwaIzPY,"[convenience_store, gas_station, food, point_o...",4,"35.83933442,-83.85799916",35.900667,-83.822994
4,"12001 Kingston Pike, Knoxville",NaN,Weigel's,{'open_now': True},"[{'height': 543, 'html_attributions': ['<a hre...",ChIJkehHkqguXIgRzGeITFuUlrs,"{'compound_code': 'VRC8+MX Knoxville, Tennesse...",4.4,ChIJkehHkqguXIgRzGeITFuUlrs,"[gas_station, atm, convenience_store, finance,...",16,"35.85753808,-84.19486739",35.871677,-84.182560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,"7100 Tazewell Pike, Corryton",NaN,Bread Box,{'open_now': True},NaN,ChIJFzCFblRrXIgRlkxmjiGnssE,"{'compound_code': '445X+R5 Corryton, Tennessee...",3.0,ChIJFzCFblRrXIgRlkxmjiGnssE,"[convenience_store, gas_station, food, point_o...",2,"36.11195017,-83.85799916",36.109524,-83.852073
62,"7415 Tazewell Pike, Corryton",NaN,Dollar General,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJE2ylNFprXIgRLlOa8IxRy8U,"{'compound_code': '449W+Q9 Corryton, Tennessee...",4.0,ChIJE2ylNFprXIgRLlOa8IxRy8U,"[drugstore, convenience_store, home_goods_stor...",332,"36.11195017,-83.85799916",36.119434,-83.854075
63,"7503 Tazewell Pike, Corryton",NaN,E-Z Stop Food Marts,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJaUAFjVtrXIgRrfyAiPO0ILE,"{'compound_code': '44CW+G9 Corryton, Tennessee...",3.3,ChIJaUAFjVtrXIgRrfyAiPO0ILE,"[convenience_store, gas_station, food, point_o...",7,"36.11195017,-83.85799916",36.121327,-83.854064
64,"10711 Rutledge Pike, Blaine",NaN,By-Lo Market,{'open_now': True},NaN,ChIJ58xAReJyXIgRC0JfRxt661A,"{'compound_code': '47HH+4J Blaine, Tennessee',...",2.6,ChIJ58xAReJyXIgRC0JfRxt661A,"[convenience_store, gas_station, food, point_o...",13,"36.13009105,-83.72325186",36.127778,-83.720951


In [34]:
# Export addresses (POI Name + Address) to a text file to be used as search strings in Google Maps
addresses = convenience["name"] + ", " + convenience["vicinity"]
addresses.to_csv('../Data/Convenience_Addresses_20220321.txt', header=False, index=False, sep='\n', mode='w')
addresses

0     Go Vols 13th St. Grocery & Deli, 310 13th St, ...
1       Doan's Market & Deli, 2539 Cecil Ave, Knoxville
2     King's Market & Deli, 612 Tipton Station Rd, K...
3           Pioneer Market, 8119 Chapman Hwy, Knoxville
4              Weigel's, 12001 Kingston Pike, Knoxville
                            ...                        
61              Bread Box, 7100 Tazewell Pike, Corryton
62         Dollar General, 7415 Tazewell Pike, Corryton
63    E-Z Stop Food Marts, 7503 Tazewell Pike, Corryton
64            By-Lo Market, 10711 Rutledge Pike, Blaine
65            Dollar General, 256 Rutledge Pike, Blaine
Length: 66, dtype: object

#### Merge stores

In [ ]:
# Load CSV of POIs (Grocery Stores) scraped from Google Maps
grocery = pd.read_csv('../Data/POIs_Grocery.csv')
grocery['Store_Type'] = len(grocery) * ['Grocery']
convenience = pd.read_csv('../Data/POIs_Convenience.csv')
convenience['Store_Type'] = len(convenience) * ['Convenience']
wholesale = pd.read_csv('../Data/POIs_Wholesale.csv')
wholesale['Store_Type'] = len(wholesale) * ['Wholesale']

# Combine store datasets
stores = pd.concat([grocery, convenience, wholesale])

# Remove duplicate entries
stores.drop_duplicates(subset=['PlusCode'], inplace=True)

# Reset index
stores.reset_index(drop=True, inplace=True)

# Extract latitude, longitude from the Google Maps search URL
stores["LatLon"] = stores["G_MAP_URL"].str.extract(r'(@\d{1,3}.\d{4,7},-?\d{1,3}.\d{4,7})')#.astype(str).replace('@', '')

import re
# Pre-compile regex pattern
p = re.compile(r'@')

# Extract latitude and longitude
stores["Latitude"], stores["Longitude"] = zip(*[p.sub('', latlon).split(',') for latlon in stores["LatLon"]])

# Set data type to float
stores["Latitude"] = stores["Latitude"].astype(float)
stores["Longitude"] = stores["Longitude"].astype(float)

# Clean fields for use in interactive visualization
stores = stores.fillna("N/A")
stores.Rating = stores.Rating.astype(str)

# Create GeoDataFrame
stores_gdf = gpd.GeoDataFrame(stores, geometry=gpd.points_from_xy(stores.Longitude, stores.Latitude))

# Set coordinate system
#epsg = 2274 # NAD83 / Tennessee State Plane (feet)
epsg = 4326
stores_gdf.set_crs(epsg=epsg, inplace=True)

# Export as shapefile
stores_gdf.to_file('../Data/All_Food_Stores_20220325.shp')

# Psuedo z-value settings for stores_all in visualization
stores_Z = []
stores_Z += len(stores) * [datetime(2022, 3, 10, 23, 59, 59)]
stores['Psuedo_Z'] = stores_Z

# Export as CSV
stores.to_csv('../Data/All_Food_Stores_20220325.csv')

In [28]:
stores = pd.read_csv('../Data/All_Food_Stores_20220325.csv')

In [29]:
from ast import literal_eval

def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

# Pandas was reading this column as a string and not a dictionary,
# so call literal_eval to convert the column from a string to dictionary
stores['Services'] = stores['Services'].apply(try_literal_eval)
# Unpack the dictionary column into individual columns
# Each column represents a main category of service options (e.g., shopping mode, health and safety, atmosphere, etc.)
# For each row (store) within each column is a list of related features for that store 
main_services = stores['Services'].apply(pd.Series)

# Create dictionary of relevant service options
service_dict = {
    'Service options':
             [
                'Has in-store shopping',
                'Offers delivery',
                'Offers curbside pickup',
                'In-store pickup for online orders',
                'Has drive-through',
                'Has no-contact delivery',
                'Offers same-day delivery'
             ],
    'Health & safety':
            [
                'Staff required to disinfect surfaces between visits',
                'Safety dividers at checkout',
                'Mask required',
                'Staff wear masks',
                'Staff get temperature checks'
            ],
    'Highlights':
            [
                'Has great service'
            ],
    'Accessibility':
            [
                'Has wheelchair accessible entrance'
            ],
    'Planning':
            [
                'Good for quick visit'
            ],
    'Offerings':
            [
                'Sells organic products',
                'Sells prepared foods'
            ],
    'Payments':
            [
                'Accepts checks',
                'Accepts debit cards',
                'Accepts NFC mobile payments',
                'Accepts SNAP/EBT',
                'Credit cards'
            ],
    'Amenities':
            [
                'Has public restroom'
            ],
    'Crowd':
            [
                'LGBTQ+ friendly',
                'Family-friendly'
            ]
}

# Create a new data frame for where the list of related features are unpacked and converted into individual columns
services = pd.DataFrame()

# Create a counter used as a temporary column label for the new data frame
col_int = 0

# For the key and value of each main category of features
for _, service_category in enumerate(service_dict):
    
    # For each secondary feature within the current main feature category
    for second_feature in service_dict[service_category]:
        
        # Create a temporary list that will record whether each row (store) contains the particular secondary feature
        serv_list = []
        
        # For each row (store) in the column of main category in the main_services data frame
        for row in main_services[service_category]:
            
            # All recorded features were stored as a list
            # So if the current row, column value is a list
            if isinstance(row, list):
                
                # If the secondary feature (as a string) is in the list
                if second_feature in row:
                    # Record yes
                    yesno = 'Yes'
                    # Append the temporary list with yes
                    serv_list += [yesno]
                # Otherwise
                else:
                    # Record no
                    yesno = ''
                    # Append the temporary list with no
                    serv_list += [yesno]
            # If the current row, column value is not a list
            else:
                # Record no
                yesno = ''
                # Append the temporary list with no
                serv_list += [yesno]

        # Store the list of Yes/No's as a column in the services data frame
        services[col_int] = serv_list
        # Increase the counter of a temporary column label name
        col_int += 1

# Delete the counter entirely
del col_int

# Rename columns in the new data frame
services.rename(columns={
    0:'Shop_Store',
    1:'Delivery',
    2:'Pickup_Curbside',
    3:'Pickup_Store',
    4:'Drive_Thru',
    5:'Delivery_No_Contact',
    6:'Delivery_Same_Day',
    7:'Disinfect_Surfaces',
    8:'Safety_Dividers_Checkout',
    9:'Masks_Required',
    10:'Masks_Staff',
    11:'Staff_Temperature_Check',
    12:'Great_Service',
    13:'Wheelchair_Accessible',
    14:'Quick_Visit',
    15:'Organic_Food',
    16:'Prepared_Food',
    17:'Pay_Checks',
    18:'Pay_Debit_Cards',
    19:'Pay_NFC_Mobile',
    20:'Pay_SNAP_EBT',
    21:'Pay_Credit_Cards',
    22:'Restroom',
    23:'LGBTQ_Friendly',
    24:'Family_Friendly',
}, inplace=True)
                
services

,Shop_Store,Delivery,Pickup_Curbside,Pickup_Store,Drive_Thru,Delivery_No_Contact,Delivery_Same_Day,Disinfect_Surfaces,Safety_Dividers_Checkout,Masks_Required,...,Organic_Food,Prepared_Food,Pay_Checks,Pay_Debit_Cards,Pay_NFC_Mobile,Pay_SNAP_EBT,Pay_Credit_Cards,Restroom,LGBTQ_Friendly,Family_Friendly
0,Yes,,,,,,,Yes,Yes,,...,,,,,,,,,,
1,Yes,,Yes,Yes,,,,Yes,Yes,,...,Yes,Yes,Yes,Yes,Yes,,,,,
2,Yes,,Yes,Yes,Yes,,,Yes,Yes,,...,Yes,Yes,Yes,Yes,Yes,Yes,,,,
3,Yes,Yes,,,,Yes,,Yes,Yes,,...,Yes,,,Yes,Yes,Yes,Yes,,,
4,Yes,,Yes,Yes,,,Yes,Yes,Yes,Yes,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Yes,,,,,,,,,,...,,,,,,Yes,,,,
217,Yes,,,,,,,,,,...,,,,,,,,,,
218,Yes,,,Yes,,,,Yes,Yes,,...,,,,,,Yes,,,,
219,Yes,,,Yes,,,,Yes,Yes,,...,,,Yes,Yes,Yes,Yes,,,,


In [68]:
# If any pickup or delivery option is available, we'll assume the store offers delivery
# Costco delivers, so manually edit the data
services.loc[219, 'Delivery'] = 'Yes'
services_cols_online = ['Delivery', 'Pickup_Curbside', 'Delivery_No_Contact', 'Delivery_Same_Day']
m1 = services[services_cols_online].eq('Yes').any(axis=1)

import numpy as np

services['Shop_Online'] = np.select([m1], [True], default=False)

In [69]:
services_cols_temp = services.columns.tolist()
services_cols_new = services_cols_temp[-1:] + services_cols_temp[:-1]
services = services[services_cols_new]
services

,LGBTQ_Friendly,Family_Friendly,Shop_Online,Shop_Store,Delivery,Pickup_Curbside,Pickup_Store,Drive_Thru,Delivery_No_Contact,Delivery_Same_Day,...,Wheelchair_Accessible,Quick_Visit,Organic_Food,Prepared_Food,Pay_Checks,Pay_Debit_Cards,Pay_NFC_Mobile,Pay_SNAP_EBT,Pay_Credit_Cards,Restroom
0,,,False,Yes,,,,,,,...,Yes,Yes,,,,,,,,
1,,,True,Yes,,Yes,Yes,,,,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,,,
2,,,True,Yes,,Yes,Yes,Yes,,,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,,
3,,,True,Yes,Yes,,,,Yes,,...,Yes,Yes,Yes,,,Yes,Yes,Yes,Yes,
4,,,True,Yes,,Yes,Yes,,,Yes,...,Yes,Yes,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,,,False,Yes,,,,,,,...,Yes,Yes,,,,,,Yes,,
217,,,False,Yes,,,,,,,...,Yes,Yes,,,,,,,,
218,,,False,Yes,,,Yes,,,,...,Yes,Yes,,,,,,Yes,,
219,,,True,Yes,Yes,,Yes,,,,...,Yes,,,,Yes,Yes,Yes,Yes,,


In [70]:
# See Reviews_Analysis.ipynb; Stores was joined to average rating for different topics extracted from reviews
store_features = pd.read_csv('../Data/All_Food_Stores_Features_20220326.csv', index_col=0)
store_features = pd.merge(store_features, services, left_index=True, right_index=True)

In [39]:
#store_features = pd.read_csv('../Data/All_Food_Stores_Features_20220327.csv')

from ast import literal_eval

def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

# Pandas was reading this column as a string and not a dictionary,
# so call literal_eval to convert the column from a string to dictionary
store_features['Hours'] = store_features['Hours'].apply(try_literal_eval)

# Unpack the dictionary column into individual columns
# Each column represents the opening and closing hours of a weekday/weekend
# For each row (store) within each column is a list of opening and closing hours of a business
open_hours = store_features['Hours'].apply(pd.Series)

# Replace all non-list objects with an empty list
for day in open_hours.columns.values:
    open_hours[day] = open_hours[day].apply(lambda d: d if isinstance(d, list) else [])
    
# Check if any column contains more than 2 values
# >2 values would indicate there are multiple opening and closing hour times
for day in open_hours.columns.values:
    if len(open_hours[open_hours[day].str.len() > 2]) > 0:
        print('Check ' + day)

open_hours

days = {
    'Tuesday': '2022/03/08',
    'Wednesday': '2022/03/09',
    'Thursday': '2022/03/10',
    'Friday': '2022/03/11',
    'Saturday': '2022/03/12',
    'Sunday': '2022/03/13',
    'Monday': '2022/03/14'
}

# Create a new data frame where unpacked opening and closing hours will be stored
open_hours_unpack = pd.DataFrame()

# For each column of a weekday or weekend in open_hours
for _, col in enumerate(open_hours):
    
    # Create new column names for the, respectively, opening and closing hours of the week(day/end)
    name_col_open = col + '_Open'
    name_col_close = col + '_Close'
    
    # Create temporary lists that will store a datetime object for the opening and closing hours of each row
    o = []
    c = []
    
    # For each row in the current column
    for row in open_hours[col]:
        # If the length of the list is greater than 1, which indicates that there is both an open and close hour
        if len(row) > 1:
            # Append a string of the day and opening hour time to o
            o.append(days[col] + ' ' + row[0])
            # Append a string of the day and closing hour time to c
            c.append(days[col] + ' ' + row[1])
        else:
            # Else, we'll assume the store is never closed
            o.append(days[col] + ' 00:00:00')
            c.append('2261/12/31' + ' 00:00:00')
    
    # Convert each list to series with datetime types
    open_hours_unpack[name_col_open] = pd.to_datetime(o)
    open_hours_unpack[name_col_close] = pd.to_datetime(c)
    
open_hours_unpack

,Tuesday_Open,Tuesday_Close,Wednesday_Open,Wednesday_Close,Thursday_Open,Thursday_Close,Friday_Open,Friday_Close,Saturday_Open,Saturday_Close,Sunday_Open,Sunday_Close,Monday_Open,Monday_Close
0,2022-03-08 08:00:00,2022-03-08 19:00:00,2022-03-09 08:00:00,2022-03-09 19:00:00,2022-03-10 08:00:00,2022-03-10 19:00:00,2022-03-11 08:00:00,2022-03-11 19:00:00,2022-03-12 08:00:00,2022-03-12 19:00:00,2022-03-13 10:00:00,2022-03-13 18:00:00,2022-03-14 08:00:00,2022-03-14 19:00:00
1,2022-03-08 06:00:00,2022-03-08 23:00:00,2022-03-09 06:00:00,2022-03-09 23:00:00,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
2,2022-03-08 06:00:00,2022-03-08 23:00:00,2022-03-09 06:00:00,2022-03-09 23:00:00,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
3,2022-03-08 08:30:00,2022-03-08 20:00:00,2022-03-09 09:00:00,2022-03-09 20:00:00,2022-03-10 08:30:00,2022-03-10 20:00:00,2022-03-11 09:00:00,2022-03-11 20:00:00,2022-03-12 09:00:00,2022-03-12 20:00:00,2022-03-13 09:00:00,2022-03-13 20:00:00,2022-03-14 09:00:00,2022-03-14 20:00:00
4,2022-03-08 06:00:00,2022-03-08 23:00:00,2022-03-09 06:00:00,2022-03-09 23:00:00,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2022-03-08 00:00:00,2261-12-31 00:00:00,2022-03-09 00:00:00,2261-12-31 00:00:00,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
217,2022-03-08 00:00:00,2261-12-31 00:00:00,2022-03-09 00:00:00,2261-12-31 00:00:00,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
218,2022-03-08 08:00:00,2022-03-08 22:00:00,2022-03-09 08:00:00,2022-03-09 22:00:00,2022-03-10 08:00:00,2022-03-10 22:00:00,2022-03-11 08:00:00,2022-03-11 22:00:00,2022-03-12 08:00:00,2022-03-12 22:00:00,2022-03-13 08:00:00,2022-03-13 22:00:00,2022-03-14 08:00:00,2022-03-14 22:00:00
219,2022-03-08 10:00:00,2022-03-08 20:30:00,2022-03-09 10:00:00,2022-03-09 20:30:00,2022-03-10 10:00:00,2022-03-10 20:30:00,2022-03-11 10:00:00,2022-03-11 20:30:00,2022-03-12 09:30:00,2022-03-12 18:00:00,2022-03-13 10:00:00,2022-03-13 18:00:00,2022-03-14 10:00:00,2022-03-14 20:30:00


In [71]:
store_features = pd.merge(store_features, open_hours_unpack, left_index=True, right_index=True)
store_features

,Name,Rating,Price,Type,Address,Hours,Website,Phone,PlusCode,Descript,...,Thursday_Open,Thursday_Close,Friday_Open,Friday_Close,Saturday_Open,Saturday_Close,Sunday_Open,Sunday_Close,Monday_Open,Monday_Close
0,United Grocery Outlet,4.3,NaN,Grocery store,"1210 E Broadway St, Lenoir City, TN 37772","{'Tuesday': ['8AM', '7PM'], 'Wednesday': ['8AM...",Website: myugo.com,(865) 986-2471,"QPXW+J6 Lenoir City, Tenn",NaN,...,2022-03-10 08:00:00,2022-03-10 19:00:00,2022-03-11 08:00:00,2022-03-11 19:00:00,2022-03-12 08:00:00,2022-03-12 19:00:00,2022-03-13 10:00:00,2022-03-13 18:00:00,2022-03-14 08:00:00,2022-03-14 19:00:00
1,Ingles Markets,4.3,$$,Grocery store,"11847 Kingston Pike, Knoxville, TN 37934","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: ingles-markets.com,(865) 966-4360,"VRFC+WR Knoxville, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
2,Ingles Markets,4.4,$$,Supermarket,"404 US-321 #1, Lenoir City, TN 37771","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: ingles-markets.com,(865) 988-5902,"RP2P+RV Lenoir City, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
3,ALDI,4.5,$,Supermarket,"725 US Hwy 321 N, Lenoir City, TN 37771","{'Tuesday': ['8:30AM', '8PM'], 'Wednesday': ['...",Website: aldi.us,(855) 955-2534,"RP6P+MW Lenoir City, Tenn","Retail chain selling a range of grocery items,...",...,2022-03-10 08:30:00,2022-03-10 20:00:00,2022-03-11 09:00:00,2022-03-11 20:00:00,2022-03-12 09:00:00,2022-03-12 20:00:00,2022-03-13 09:00:00,2022-03-13 20:00:00,2022-03-14 09:00:00,2022-03-14 20:00:00
4,Walmart Supercenter,3.9,$,Department store,"911 Highway 321 N, Lenoir City, TN 37771","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: walmart.com,(865) 986-9002,"RPCP+R5 Lenoir City, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,E-Z Stop Food Marts,3.0,NaN,Convenience store,"7503 Tazewell Pike, Corryton, TN 37721","{'Monday': ['Open 24 hours'], 'Tuesday': ['Ope...",Website: ezstop.net,(865) 689-2688,"44CW+G9 Corryton, Tenn",NaN,...,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
217,By-Lo Market,2.6,NaN,Convenience store,"10711 Rutledge Pike, Blaine, TN 37709","{'Monday': ['Open 24 hours'], 'Tuesday': ['Ope...",NaN,(865) 933-2028,"47HH+4J Blaine, Tenn",NaN,...,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
218,Dollar General,4.1,$,Dollar store,"256 Rutledge Pike, Blaine, TN 37709","{'Monday': ['8AM', '10PM'], 'Tuesday': ['8AM',...",Website: stores.dollargeneral.com,(865) 401-0029,"47PP+P4 Blaine, Tenn",Bargain retail chain selling a range of househ...,...,2022-03-10 08:00:00,2022-03-10 22:00:00,2022-03-11 08:00:00,2022-03-11 22:00:00,2022-03-12 08:00:00,2022-03-12 22:00:00,2022-03-13 08:00:00,2022-03-13 22:00:00,2022-03-14 08:00:00,2022-03-14 22:00:00
219,Costco Wholesale,4.6,$$,Warehouse store,"10745 Kingston Pike, Knoxville, TN 37934","{'Friday': ['10AM', '8:30PM'], 'Saturday': ['9...",Website: costco.com,(865) 218-7700,"VVW5+MQ Knoxville, Tenn",Members-only warehouse selling a huge variety ...,...,2022-03-10 10:00:00,2022-03-10 20:30:00,2022-03-11 10:00:00,2022-03-11 20:30:00,2022-03-12 09:30:00,2022-03-12 18:00:00,2022-0

In [72]:
store_features.columns.values

array(['Name', 'Rating', 'Price', 'Type', 'Address', 'Hours', 'Website',
       'Phone', 'PlusCode', 'Descript', 'Services', 'G_MAP_URL',
       'PopTimes', 'LatLon', 'Latitude', 'Longitude', 'geometry',
       'Psuedo_Z', 'Convenience', 'Customer Service and Checkout Process',
       'Employees', 'Food Departments and Quality',
       'Food Selection and Variety', 'Price and General Experience',
       'LGBTQ_Friendly', 'Family_Friendly', 'Shop_Online', 'Shop_Store',
       'Delivery', 'Pickup_Curbside', 'Pickup_Store', 'Drive_Thru',
       'Delivery_No_Contact', 'Delivery_Same_Day', 'Disinfect_Surfaces',
       'Safety_Dividers_Checkout', 'Masks_Required', 'Masks_Staff',
       'Staff_Temperature_Check', 'Great_Service',
       'Wheelchair_Accessible', 'Quick_Visit', 'Organic_Food',
       'Prepared_Food', 'Pay_Checks', 'Pay_Debit_Cards', 'Pay_NFC_Mobile',
       'Pay_SNAP_EBT', 'Pay_Credit_Cards', 'Restroom', 'Tuesday_Open',
       'Tuesday_Close', 'Wednesday_Open', 'Wednesday_Close

In [73]:
store_features_gdf = gpd.GeoDataFrame(
        store_features.drop(['LatLon', 'Services'], axis=1),
        crs={'init': 'epsg:4326'},
        geometry=[Point(xy) for xy in zip(store_features.Longitude, store_features.Latitude)])
store_features_gdf

C:\Users\jfeng\Anaconda3\envs\hagerstrand\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Name,Rating,Price,Type,Address,Hours,Website,Phone,PlusCode,Descript,...,Thursday_Open,Thursday_Close,Friday_Open,Friday_Close,Saturday_Open,Saturday_Close,Sunday_Open,Sunday_Close,Monday_Open,Monday_Close
0,United Grocery Outlet,4.3,NaN,Grocery store,"1210 E Broadway St, Lenoir City, TN 37772","{'Tuesday': ['8AM', '7PM'], 'Wednesday': ['8AM...",Website: myugo.com,(865) 986-2471,"QPXW+J6 Lenoir City, Tenn",NaN,...,2022-03-10 08:00:00,2022-03-10 19:00:00,2022-03-11 08:00:00,2022-03-11 19:00:00,2022-03-12 08:00:00,2022-03-12 19:00:00,2022-03-13 10:00:00,2022-03-13 18:00:00,2022-03-14 08:00:00,2022-03-14 19:00:00
1,Ingles Markets,4.3,$$,Grocery store,"11847 Kingston Pike, Knoxville, TN 37934","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: ingles-markets.com,(865) 966-4360,"VRFC+WR Knoxville, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
2,Ingles Markets,4.4,$$,Supermarket,"404 US-321 #1, Lenoir City, TN 37771","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: ingles-markets.com,(865) 988-5902,"RP2P+RV Lenoir City, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
3,ALDI,4.5,$,Supermarket,"725 US Hwy 321 N, Lenoir City, TN 37771","{'Tuesday': ['8:30AM', '8PM'], 'Wednesday': ['...",Website: aldi.us,(855) 955-2534,"RP6P+MW Lenoir City, Tenn","Retail chain selling a range of grocery items,...",...,2022-03-10 08:30:00,2022-03-10 20:00:00,2022-03-11 09:00:00,2022-03-11 20:00:00,2022-03-12 09:00:00,2022-03-12 20:00:00,2022-03-13 09:00:00,2022-03-13 20:00:00,2022-03-14 09:00:00,2022-03-14 20:00:00
4,Walmart Supercenter,3.9,$,Department store,"911 Highway 321 N, Lenoir City, TN 37771","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: walmart.com,(865) 986-9002,"RPCP+R5 Lenoir City, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,E-Z Stop Food Marts,3.0,NaN,Convenience store,"7503 Tazewell Pike, Corryton, TN 37721","{'Monday': ['Open 24 hours'], 'Tuesday': ['Ope...",Website: ezstop.net,(865) 689-2688,"44CW+G9 Corryton, Tenn",NaN,...,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
217,By-Lo Market,2.6,NaN,Convenience store,"10711 Rutledge Pike, Blaine, TN 37709","{'Monday': ['Open 24 hours'], 'Tuesday': ['Ope...",NaN,(865) 933-2028,"47HH+4J Blaine, Tenn",NaN,...,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
218,Dollar General,4.1,$,Dollar store,"256 Rutledge Pike, Blaine, TN 37709","{'Monday': ['8AM', '10PM'], 'Tuesday': ['8AM',...",Website: stores.dollargeneral.com,(865) 401-0029,"47PP+P4 Blaine, Tenn",Bargain retail chain selling a range of househ...,...,2022-03-10 08:00:00,2022-03-10 22:00:00,2022-03-11 08:00:00,2022-03-11 22:00:00,2022-03-12 08:00:00,2022-03-12 22:00:00,2022-03-13 08:00:00,2022-03-13 22:00:00,2022-03-14 08:00:00,2022-03-14 22:00:00
219,Costco Wholesale,4.6,$$,Warehouse store,"10745 Kingston Pike, Knoxville, TN 37934","{'Friday': ['10AM', '8:30PM'], 'Saturday': ['9...",Website: costco.com,(865) 218-7700,"VVW5+MQ Knoxville, Tenn",Members-only warehouse selling a huge variety ...,...,2022-03-10 10:00:00,2022-03-10 20:30:00,2022-03-11 10:00:00,2022-03-11 20:30:00,2022-03-12 09:30:00,2022-03-12 18:00:00,2022-0

In [13]:
# https://github.com/m-wrzr/populartimes/blob/master/populartimes/crawler.py
# See above for more inspiration
from datetime import datetime

def flatten_list(t):
    temp = [item for sublist in t for item in sublist]
    return temp

days = {
    'Tuesday': '2022/03/08',
    'Wednesday': '2022/03/09',
    'Thursday': '2022/03/10',
    'Friday': '2022/03/11',
    'Saturday': '2022/03/12',
    'Sunday': '2022/03/13',
    'Monday': '2022/03/14'
}

def flatten_to_dict(t):
    temp = {datetime.strptime(days[item[0]] + ', ' + item[1], '%Y/%m/%d, %I %p') : item[2] for item in t}
    return temp

temp_pop_dict = flatten_to_dict(store_features_gdf.loc[0,'PopTimes'])
[item[0] for item in sorted(temp_pop_dict.items(), key=lambda val: val[1])]

[datetime.datetime(2022, 3, 13, 6, 0),
 datetime.datetime(2022, 3, 13, 7, 0),
 datetime.datetime(2022, 3, 13, 8, 0),
 datetime.datetime(2022, 3, 13, 9, 0),
 datetime.datetime(2022, 3, 8, 18, 0),
 datetime.datetime(2022, 3, 8, 19, 0),
 datetime.datetime(2022, 3, 8, 20, 0),
 datetime.datetime(2022, 3, 8, 21, 0),
 datetime.datetime(2022, 3, 9, 22, 0),
 datetime.datetime(2022, 3, 9, 23, 0),
 datetime.datetime(2022, 3, 9, 6, 0),
 datetime.datetime(2022, 3, 9, 7, 0),
 datetime.datetime(2022, 3, 11, 19, 0),
 datetime.datetime(2022, 3, 11, 20, 0),
 datetime.datetime(2022, 3, 11, 21, 0),
 datetime.datetime(2022, 3, 12, 22, 0),
 datetime.datetime(2022, 3, 12, 23, 0),
 datetime.datetime(2022, 3, 12, 6, 0),
 datetime.datetime(2022, 3, 12, 7, 0),
 datetime.datetime(2022, 3, 14, 19, 0),
 datetime.datetime(2022, 3, 14, 20, 0),
 datetime.datetime(2022, 3, 14, 21, 0),
 datetime.datetime(2022, 3, 8, 22, 0),
 datetime.datetime(2022, 3, 8, 23, 0),
 datetime.datetime(2022, 3, 8, 6, 0),
 datetime.datetime(2

In [74]:
store_features_gdf.drop('PopTimes', axis=1).to_csv('../Data/All_Food_Stores_Features.csv')
#store_features_gdf.drop('PopTimes', axis=1).to_file('../Data/All_Food_Stores_Features.shp', schema=schema)

In [75]:
store_features_gdf

,Name,Rating,Price,Type,Address,Hours,Website,Phone,PlusCode,Descript,...,Thursday_Open,Thursday_Close,Friday_Open,Friday_Close,Saturday_Open,Saturday_Close,Sunday_Open,Sunday_Close,Monday_Open,Monday_Close
0,United Grocery Outlet,4.3,NaN,Grocery store,"1210 E Broadway St, Lenoir City, TN 37772","{'Tuesday': ['8AM', '7PM'], 'Wednesday': ['8AM...",Website: myugo.com,(865) 986-2471,"QPXW+J6 Lenoir City, Tenn",NaN,...,2022-03-10 08:00:00,2022-03-10 19:00:00,2022-03-11 08:00:00,2022-03-11 19:00:00,2022-03-12 08:00:00,2022-03-12 19:00:00,2022-03-13 10:00:00,2022-03-13 18:00:00,2022-03-14 08:00:00,2022-03-14 19:00:00
1,Ingles Markets,4.3,$$,Grocery store,"11847 Kingston Pike, Knoxville, TN 37934","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: ingles-markets.com,(865) 966-4360,"VRFC+WR Knoxville, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
2,Ingles Markets,4.4,$$,Supermarket,"404 US-321 #1, Lenoir City, TN 37771","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: ingles-markets.com,(865) 988-5902,"RP2P+RV Lenoir City, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
3,ALDI,4.5,$,Supermarket,"725 US Hwy 321 N, Lenoir City, TN 37771","{'Tuesday': ['8:30AM', '8PM'], 'Wednesday': ['...",Website: aldi.us,(855) 955-2534,"RP6P+MW Lenoir City, Tenn","Retail chain selling a range of grocery items,...",...,2022-03-10 08:30:00,2022-03-10 20:00:00,2022-03-11 09:00:00,2022-03-11 20:00:00,2022-03-12 09:00:00,2022-03-12 20:00:00,2022-03-13 09:00:00,2022-03-13 20:00:00,2022-03-14 09:00:00,2022-03-14 20:00:00
4,Walmart Supercenter,3.9,$,Department store,"911 Highway 321 N, Lenoir City, TN 37771","{'Tuesday': ['6AM', '11PM'], 'Wednesday': ['6A...",Website: walmart.com,(865) 986-9002,"RPCP+R5 Lenoir City, Tenn",NaN,...,2022-03-10 06:00:00,2022-03-10 23:00:00,2022-03-11 06:00:00,2022-03-11 23:00:00,2022-03-12 06:00:00,2022-03-12 23:00:00,2022-03-13 06:00:00,2022-03-13 23:00:00,2022-03-14 06:00:00,2022-03-14 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,E-Z Stop Food Marts,3.0,NaN,Convenience store,"7503 Tazewell Pike, Corryton, TN 37721","{'Monday': ['Open 24 hours'], 'Tuesday': ['Ope...",Website: ezstop.net,(865) 689-2688,"44CW+G9 Corryton, Tenn",NaN,...,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
217,By-Lo Market,2.6,NaN,Convenience store,"10711 Rutledge Pike, Blaine, TN 37709","{'Monday': ['Open 24 hours'], 'Tuesday': ['Ope...",NaN,(865) 933-2028,"47HH+4J Blaine, Tenn",NaN,...,2022-03-10 00:00:00,2261-12-31 00:00:00,2022-03-11 00:00:00,2261-12-31 00:00:00,2022-03-12 00:00:00,2261-12-31 00:00:00,2022-03-13 00:00:00,2261-12-31 00:00:00,2022-03-14 00:00:00,2261-12-31 00:00:00
218,Dollar General,4.1,$,Dollar store,"256 Rutledge Pike, Blaine, TN 37709","{'Monday': ['8AM', '10PM'], 'Tuesday': ['8AM',...",Website: stores.dollargeneral.com,(865) 401-0029,"47PP+P4 Blaine, Tenn",Bargain retail chain selling a range of househ...,...,2022-03-10 08:00:00,2022-03-10 22:00:00,2022-03-11 08:00:00,2022-03-11 22:00:00,2022-03-12 08:00:00,2022-03-12 22:00:00,2022-03-13 08:00:00,2022-03-13 22:00:00,2022-03-14 08:00:00,2022-03-14 22:00:00
219,Costco Wholesale,4.6,$$,Warehouse store,"10745 Kingston Pike, Knoxville, TN 37934","{'Friday': ['10AM', '8:30PM'], 'Saturday': ['9...",Website: costco.com,(865) 218-7700,"VVW5+MQ Knoxville, Tenn",Members-only warehouse selling a huge variety ...,...,2022-03-10 10:00:00,2022-03-10 20:30:00,2022-03-11 10:00:00,2022-03-11 20:30:00,2022-03-12 09:30:00,2022-03-12 18:00:00,2022-0